In [1]:
import pickle as pickle

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

tf_idf_ques_rec = load_obj("tf_idf_ques_rec")
recommended = tf_idf_ques_rec

In [2]:
import pandas as pd
postlinksDF = pd.read_csv('postlinks.csv')

In [3]:
postlinksDF.head()

,Id,PostId,RelatedPostId,LinkTypeId
0,865,155,143,1
1,891,143,155,1
2,1139,192,57,1
3,1789,245,256,1
4,2699,341,302,1


In [5]:
import tqdm 
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in tqdm.tqdm(postlinksDF.iterrows()):
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)

9855it [01:44, 94.60it/s] 


In [6]:
len(postLinks)

6579

In [7]:
def evaluate():
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']]:
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.3* (present/count)+ 0.7*(top/present))
    return (score/len(postLinks))

In [8]:
evaluate()

0.0